# POSTTEST 6 KECERDASAN BUATAN
*Maulana Muhammad Hafidz / 2109106070*

Pada posttest 6 ini saya mengganti dataset dengan link : https://www.kaggle.com/datasets/rodolfomendes/abalone-dataset

Target dataset : Rings

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("abalone-Copy1.csv")
df

Sex  Length  Diameter  Height  Whole weight  Shucked weight  \
0      M   0.455     0.365   0.095        0.5140          0.2245   
1      M   0.350     0.265   0.090        0.2255          0.0995   
2      F   0.530     0.420   0.135        0.6770          0.2565   
3      M   0.440     0.365   0.125        0.5160          0.2155   
4      I   0.330     0.255   0.080        0.2050          0.0895   
...   ..     ...       ...     ...           ...             ...   
4172   F   0.565     0.450   0.165        0.8870          0.3700   
4173   M   0.590     0.440   0.135        0.9660          0.4390   
4174   M   0.600     0.475   0.205        1.1760          0.5255   
4175   F   0.625     0.485   0.150        1.0945          0.5310   
4176   M   0.710     0.555   0.195        1.9485          0.9455   

      Viscera weight  Shell weight  Rings  
0             0.1010        0.1500     15  
1             0.0485        0.0700      7  
2             0.1415        0.2100      9  
3             0.1140        0.1550     10  
4             0.0395        0.0550      7  
...              ...           ...    ...  
4172          0.2390        0.2490     11  
4173          0.2145        0.2605     10  
4174          0.2875        0.3080      9  
4175          0.2610        0.2960     10  
4176          0.3765        0.4950     12  

[4177 rows x 9 columns]

Mengkonversi target ke binary

In [8]:
df['Rings'] = df['Rings'].apply(lambda x: 1 if x > 10 else 0)

Pisahkan Feature dan Target

In [7]:
X = df[['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Sex']]
y = df['Rings']

Melakukan encoding pada kolom "Sex"

In [10]:
X = pd.get_dummies(X, columns=['Sex'], prefix='Sex', drop_first=True)

Bagi dataset menjadi Train dan Test

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Standarisasi pada kolom feature

In [98]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Membuat model squential

In [99]:
model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [100]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 64)                640       
                                                                 
 dense_31 (Dense)            (None, 32)                2080      
                                                                 
 dense_32 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2753 (10.75 KB)
Trainable params: 2753 (10.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Mengompilasi model

In [128]:
model.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])

In [129]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)

Melatih Model

In [130]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, checkpoint])

Epoch 1/50
84/84 [==============================] - 2s 7ms/step - loss: 0.3951 - accuracy: 0.8114 - val_loss: 0.4423 - val_accuracy: 0.7848
Epoch 2/50
84/84 [==============================] - 0s 4ms/step - loss: 0.3921 - accuracy: 0.8147 - val_loss: 0.4491 - val_accuracy: 0.7833
Epoch 3/50
84/84 [==============================] - 0s 4ms/step - loss: 0.3930 - accuracy: 0.8099 - val_loss: 0.4503 - val_accuracy: 0.7848
Epoch 4/50
84/84 [==============================] - 0s 4ms/step - loss: 0.3914 - accuracy: 0.8136 - val_loss: 0.4402 - val_accuracy: 0.7877
Epoch 5/50
84/84 [==============================] - 0s 4ms/step - loss: 0.3948 - accuracy: 0.8118 - val_loss: 0.4414 - val_accuracy: 0.7833
Epoch 6/50
84/84 [==============================] - 0s 4ms/step - loss: 0.3922 - accuracy: 0.8121 - val_loss: 0.4462 - val_accuracy: 0.7877
Epoch 7/50
84/84 [==============================] - 0s 3ms/step - loss: 0.3962 - accuracy: 0.8121 - val_loss: 0.4485 - val_accuracy: 0.7788
Epoch 8/50
84/84 [==

In [132]:
score = model.evaluate(X_test, y_test, verbose=1)

27/27 [==============================] - 0s 2ms/step - loss: 0.4315 - accuracy: 0.7859


In [133]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.show()
